<h1> Auvik - API - Monthly Config Change Report </h1>

# DataFrame Creation from API
## Prepare for DataFrame Creation


# Import Modules and Define Globals

In [ ]:
# data import and file manipulation
import requests
from requests.structures import CaseInsensitiveDict
import json
import csv
import xlrd

#data conditioning
import pandas as pd
import numpy as np
import re
import datetime as dt

#data visualization
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# add current timestamp to filename for reference
current_time = (dt.datetime.utcnow().strftime('%Y_%m_%d_%H%M%S'))

# git repo folder
git_folder = 'd:/git/example_infrastructure_data_dev'

# dictionary Directory
dictionary_dir = 'd:/git/example_infrastructure_data_dev/dictionaries'

# export folder will contain all csv exported DataFrames for Ticket Creation
export_folder = 'd:/exports/'

## Create env Variables

In [ ]:
# import configparser for env secrets
from configparser import ConfigParser

config = ConfigParser()
config.read(f'{git_folder}/config/env.ini')
import requests
from requests.structures import CaseInsensitiveDict

In [ ]:
# import and assign secrets from env.ini

auvik_config = config['auvik']

## Verify permissions in Auvik


In [ ]:
# verify permissions URL

request_uri = f"{auvik_config['base_uri']}/authentication/verify"

# construct header
headers = CaseInsensitiveDict()
headers['Authorization'] = f"Basic {auvik_config['api_key']}"
#headers['auth'] = f'{username,basic_auth}'

# construct req body
data = ""

# request content response (200) is GOOD
resp = requests.get(request_uri, auth=(auvik_config['username'],auvik_config['api_key']), headers=headers)
print(resp.status_code)
print(resp)

## Pull Tenant ID

In [ ]:
# request uri
request_uri = f"{auvik_config['base_uri']}/tenants"

# construct header
headers = CaseInsensitiveDict()
headers['Authorization'] = f"Basic {auvik_config['api_key']}"

# construct req body
data = CaseInsensitiveDict()

# request content response
resp = requests.get(request_uri,  auth=(auvik_config['username'],auvik_config['api_key']), headers=headers)
content = resp.content.decode('utf-8')
c_dict = json.loads(content)

# count pages
total_pages = 0

df_auvik_tenants = pd.DataFrame(c_dict['data'])

In [ ]:
def domainPrefix(attributes):
    return attributes['domainPrefix']

def tenantType(attributes):
    return attributes['tenantType']

df_auvik_tenants['domainPrefix'] = df_auvik_tenants['attributes'].apply(domainPrefix)
df_auvik_tenants['tenantType'] = df_auvik_tenants['attributes'].apply(tenantType)

df_auvik_tenants.drop(['attributes','relationships'],inplace=True,axis=1)

In [ ]:
multiclient_list = []
for index,row in df_auvik_tenants.iterrows():

    if row['tenantType'] == 'multiClient':
        multiclient_list.append(row['id'])


# Pull Configuration Change Information

In [ ]:
# request uri
request_uri = f"{auvik_config['base_uri']}/inventory/configuration"

# construct header
headers = CaseInsensitiveDict()
headers['Authorization'] = f"Basic {auvik_config['api_key']}"

# construct req body
data = CaseInsensitiveDict()
data['tenants'] = multiclient_list[0]

# request content response
resp = requests.get(request_uri,  auth=(auvik_config['username'],auvik_config['api_key']), headers=headers)
content = resp.content.decode('utf-8')
c_dict = json.loads(content)

# count pages
total_pages = 0

df_auvik_configurations = pd.DataFrame(c_dict['data'])

while c_dict['links']['next']:
    next_page = c_dict['links']['next']
    resp = requests.get(next_page,  auth=(auvik_config['username'],auvik_config['api_key']), headers=headers)
    content = resp.content.decode('utf-8')
    c_dict = json.loads(content)
    df_current_page = pd.DataFrame(c_dict['data'])
    df_auvik_configurations = pd.concat([df_auvik_configurations,df_current_page],ignore_index=False)
    try:
        print(c_dict['links']['next'])
        total_pages += 1
    except:
        print('*'* 200)
        print('End of API Pages')
        print(f'Total Pages: {total_pages}')
        break

# Shape DataFrame

In [ ]:
def backupTime(attributes):
    return attributes['backupTime']

def isRunning(attributes):
    return attributes['isRunning']

In [ ]:
df_auvik_configurations['backupTime'] = df_auvik_configurations['attributes'].apply(backupTime)
df_auvik_configurations['isRunning'] = df_auvik_configurations['attributes'].apply(isRunning)

In [ ]:
def domainPrefix(relationships):
    return relationships['tenant']['data']['attributes']['domainPrefix']

df_auvik_configurations['domainPrefix'] = df_auvik_configurations['relationships'].apply(domainPrefix)

In [ ]:
def device(relationships):
    return relationships['device']['data']['attributes']['deviceName']

df_auvik_configurations['deviceName'] =  df_auvik_configurations['relationships'].apply(device)

In [ ]:
def selfLink(links):
    return links['self']

df_auvik_configurations['selfLink'] =  df_auvik_configurations['links'].apply(selfLink)

In [ ]:
df_auvik_configurations.drop(['attributes','relationships','links'],inplace=True,axis=1)

In [ ]:
## Convert Timezone to DateTij0me UTC
df_data_timefix = pd.to_datetime(df_auvik_configurations['backupTime'], unit='ns', errors='coerce')
df_auvik_configurations['backupTime'] = df_data_timefix.values.astype('datetime64[s]')

## Standardize Client Names

In [ ]:
df = pd.read_csv(f'{dictionary_dir}/client_name_standardization.dict',delimiter='\t')
client_rename_dict = {}
for index, row in df.iterrows():
    previousName = row['previousName']
    currentName = row['currentName']
    client_rename_dict[previousName] = currentName

In [ ]:
def client_names(c_name):
    dict_length = len(client_rename_dict)
    for k, v in client_rename_dict.items():
        try:
            result = re.sub(k, v, c_name)
            if result != c_name:
                # print(f'Keyword found: {k}')
                # print(f'Replacment value: {v}')
                # print('\n')
                return v
                break
        except Exception as e:
            print(e)
            break
    return c_name

In [ ]:
df_auvik_configurations['clientName'] = df_auvik_configurations['domainPrefix'].apply(client_names)

In [ ]:
df_auvik_configurations_last_30days = df_auvik_configurations[df_auvik_configurations['backupTime'] > (dt.datetime.now() - dt.timedelta(days=30))].sort_values('dayOccurred')

# Export DataFrames and Graphics

In [ ]:
df_auvik_configurations.to_csv(export_folder + 'auvik_device_backup_log_' + str(current_time) + '.csv',index=False)
df_auvik_configurations_last_30days.to_csv(export_folder + 'service_auvik_device_backup_log_last_30days_' + str(current_time) + '.csv',index=False)

In [ ]:
df_auvik_configurations_last_30days['dayOccurred'] = df_auvik_configurations_last_30days.iloc[:]['backupTime'].dt.to_period('d')
df_auvik_configurations['monthOccured'] = df_auvik_configurations.iloc[:]['backupTime'].dt.to_period('m')

In [ ]:
# Total Device Changes
fig = plt.figure(figsize=(60,40))
sns.countplot(data=df_auvik_configurations.sort_values('monthOccured'),x = 'monthOccured', hue='clientName')
fig.savefig(export_folder + 'auvik_device_backup_log_' + str(current_time) + '.png')

In [ ]:
# Device Changes Last 30 Days
fig = plt.figure(figsize=(60,40))
sns.countplot(data=df_auvik_configurations_last_30days,x = 'dayOccurred', hue='clientName')
fig.savefig(export_folder + 'service_auvik_device_backup_log_last_30days_' + str(current_time) + '.png')